In [ ]:
!mkdir train
!unzip /content/drive/My\ Drive/Vehicle\ Janata\ Hack/train_SOaYf6m.zip -d train

In [ ]:
"/content/train/train.csv"

'/content/train/train.csv'

In [ ]:
import pandas as pd
import os
import cv2 as cv

train=pd.read_csv("/content/train/train.csv")

num_class=train["emergency_or_not"].value_counts()

classes=train["emergency_or_not"].unique()

for i in classes:
  if(i==0):
    os.mkdir(os.path.sep.join(['/content/train','not_emergency']))
  else:
    os.mkdir(os.path.sep.join(['/content/train','emergency']))
    
base='/content/train/images'

imgids=train["image_names"]

for i,v in enumerate(imgids):
    imagePath=os.path.sep.join([base,v])
    img=cv.imread(imagePath)
    if(train.iloc[i][1]==0):
      folder='not_emergency'
    else:
      folder='emergency'
    cv.imwrite(os.path.sep.join(['/content/train',folder+'/'+v]),img)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

emergency_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    zoom_range=0.15,
    rescale=1/255.,
    horizontal_flip=True,
    validation_split=0.2
)

In [ ]:
emergency_img_ids=train[train["emergency_or_not"]==1]

In [ ]:
emergency_img_ids=(emergency_img_ids)["image_names"]

In [ ]:
emergency_img_ids=list(emergency_img_ids)

In [ ]:
not_emergency_img_ids=train[train["emergency_or_not"]==0]

In [ ]:
not_emergency_img_ids=not_emergency_img_ids["image_names"]

In [ ]:
not_emergency_img_ids=list(not_emergency_img_ids)

In [ ]:
test_em=emergency_img_ids[:int(0.1*len(emergency_img_ids))]
test_not_em=not_emergency_img_ids[:int(0.1*len(not_emergency_img_ids))]

In [ ]:
emergency_img_ids=emergency_img_ids[int(0.1*len(emergency_img_ids)):]
not_emergency_img_ids=not_emergency_img_ids[int(0.1*len(not_emergency_img_ids)):]

In [ ]:
len(emergency_img_ids)

613

In [ ]:
len(test_em)

68

In [ ]:
len(not_emergency_img_ids)

869

In [ ]:
len(test_not_em)

96

In [ ]:
import numpy as np
for img in not_emergency_img_ids:
  total=0
  imgpath=os.path.sep.join(['/content/train/images',img])
  image = load_img(imgpath)
  image = img_to_array(image)
  image = np.expand_dims(image, axis=0)
  imageGen = emergency_datagen.flow(image,
                                    batch_size=1,
                                    save_to_dir='/content/train/not_emergency',
	                                  save_prefix="aug", 
                                    save_format="jpg")
  for auged_img in imageGen:
      break

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


In [ ]:
len(os.listdir('/content/train/not_emergency'))

1791

In [ ]:
import numpy as np
for img in emergency_img_ids:
  total=0
  imgpath=os.path.sep.join(['/content/train/images',img])
  image = load_img(imgpath)
  image = img_to_array(image)
  image = np.expand_dims(image, axis=0)
  imageGen = emergency_datagen.flow(image,
                                    batch_size=1,
                                    save_to_dir='/content/train/emergency',
	                                  save_prefix="aug", 
                                    save_format="jpg")
  for auged_img in imageGen:
    total+=1
    if(total>2):
      break

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


In [ ]:
len(os.listdir('/content/train/emergency'))

2839

In [ ]:
import shutil
for img in os.listdir('/content/train/not_emergency'):
  shutil.copy(os.path.sep.join(['/content/train/not_emergency',img]),'/content/drive/My Drive/Vehicle Janata Hack/Aug_emergency')

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input
import cv2

data=[] #Holds features
#Augmenting Not emergency images
for i,j in enumerate(os.listdir('/content/train/not_emergency')):
    direc = '/content/train/not_emergency/%s'%j
    img1 = cv2.imread(direc)
    img1 = cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)
    image = cv2.resize(img1,(224,224))
    image=img_to_array(image)
    image=preprocess_input(image)
    data.append(image)
    
data = np.array(data, dtype="float32")


In [ ]:
data.shape#When data was numpy array

In [ ]:
data=list(data) #Helpful while appending Emergency images

In [ ]:
#Augmenting emergency images
for i,j in enumerate(os.listdir('/content/train/emergency')):
    direc = '/content/train/emergency/%s'%j
    img1 = cv2.imread(direc)
    img1 = cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)
    image = cv2.resize(img1,(224,224))
    image=img_to_array(image)
    image=preprocess_input(image)
    data.append(image)
    
data = np.array(data, dtype="float32")

In [ ]:
labels=['no']*len(os.listdir('/content/train/not_emergency')) + ['yes']*len(os.listdir('/content/train/emergency'))

In [ ]:
labels=np.array(labels)

In [ ]:
from sklearn.preprocessing import LabelBinarizer
lb=LabelBinarizer()
labs=lb.fit_transform(labels)

In [ ]:
lb.classes_

array(['no', 'yes'], dtype='<U3')

**LOADING TEST DATA**

In [ ]:
test_data=[]
test_labels=[]
#Augmenting Not emergency test images
for i,j in enumerate(test_not_em):
    direc = '/content/train/images/%s'%j
    img1 = cv2.imread(direc)
    img1 = cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)
    image = cv2.resize(img1,(224,224))
    image=img_to_array(image)
    image=preprocess_input(image)
    test_data.append(image)

In [ ]:

#Augmenting emergency test images
for i,j in enumerate(test_em):
    direc = '/content/train/images/%s'%j
    img1 = cv2.imread(direc)
    img1 = cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)
    image = cv2.resize(img1,(224,224))
    image=img_to_array(image)
    image=preprocess_input(image)
    test_data.append(image)

test_labels=['no']*len(test_not_em) + ['yes']*len(test_em)
    

In [ ]:
#LABEL BINARIZING TEST LABELS
lb_test=LabelBinarizer()
test_lab=lb_test.fit_transform(test_labels)

In [ ]:
lb_test.classes_

array(['no', 'yes'], dtype='<U3')

**RANDOM SHUFFLING DATA**

In [ ]:
#Train data
map_data=list(zip(data,labels))
random.seed(4)
random.shuffle(map_data)
data,labels=zip(*map_data)

In [ ]:
#Checking if no. of samples is same
data=np.array(data)
labels=list(labels)
data.shape , len(labels)

((4630, 224, 224, 3), 4630)

In [ ]:
#Test data
map_data=list(zip(test_data,test_labels))
random.seed(4)
random.shuffle(map_data)
test_data,test_labels=zip(*map_data)

In [ ]:
test_data=np.array(test_data)
test_labels=list(test_labels)
test_data.shape , len(test_labels)

((164, 224, 224, 3), 164)

In [ ]:
#Optional : To be used if train and test data are both augmented
"""
from sklearn.model_selection import train_test_split
train,test,train_lab,test_lab=train_test_split(data,labs,test_size=0.2,stratify=labs,random_state=42)
"""

In [ ]:
#len(train) , len(train_lab),len(test),len(test_lab)

(3080, 3080, 771, 771)

In [ ]:
#Forgot to lab_bin_ise train labels before :p
#So gonna do it now!
lb_again=LabelBinarizer()
labels=lb_again.fit_transform(labels)
labels[:5]
#Test labels have been forgotten as well
lb_again=LabelBinarizer()
test_labels=lb_again.fit_transform(test_labels)
test_labels[:5]

In [ ]:
from tensorflow.keras.utils import to_categorical
train_labels=to_categorical(labels,num_classes=2)
test_labels_1=to_categorical(test_labels,num_classes=2)

In [ ]:
type(train_labels) , train_labels.shape , type(test_labels_1) , test_labels_1.shape 


(numpy.ndarray, (4630, 2), numpy.ndarray, (164, 2))

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.layers import Input

baseModel=ResNet50(
    include_top=False,
     weights='imagenet',
    input_shape=(224,224,3)
)

In [ ]:
x=baseModel.output
x=tf.keras.layers.AveragePooling2D(pool_size=(7,7))(x)
x=tf.keras.layers.Flatten()(x)
x=tf.keras.layers.Dense(128,activation='relu')(x)
x=tf.keras.layers.Dropout(0.5)(x)
x=tf.keras.layers.Dense(2,activation='softmax')(x)

model=tf.keras.models.Model(inputs=baseModel.input,outputs=x)

In [ ]:
init_lr=1e-5
EPOCHS=30

In [ ]:
import tensorflow
class SnapshotCallbackBuilder:
    def __init__(self, nb_epochs, nb_snapshots, init_lr=0.1):
        self.T = nb_epochs
        self.M = nb_snapshots
        self.alpha_zero = init_lr

    def get_callbacks(self, model_prefix='Model'):

        callback_list = [
            swa,
            tensorflow.keras.callbacks.LearningRateScheduler(schedule=self._cosine_anneal_schedule)
        ]

        return callback_list

    def _cosine_anneal_schedule(self, t):
        cos_inner = np.pi * (t % (self.T // self.M))  # t - 1 is used when t has 1-based indexing.
        cos_inner /= self.T // self.M
        cos_out = np.cos(cos_inner) + 1
        return float(self.alpha_zero / 2 * cos_out)

class SWA(tensorflow.keras.callbacks.Callback):
    
    def __init__(self, filepath, swa_epoch):
        super(SWA, self).__init__()
        self.filepath = filepath
        self.swa_epoch = swa_epoch 
    
    def on_train_begin(self, logs=None):
        self.nb_epoch = self.params['epochs']
        print('Stochastic weight averaging selected for last {} epochs.'
              .format(self.nb_epoch - self.swa_epoch))
        
    def on_epoch_end(self, epoch, logs=None):
        
        if epoch == self.swa_epoch:
            self.swa_weights = self.model.get_weights()
            
        elif epoch > self.swa_epoch:    
            for i in range(len(self.swa_weights)):
                self.swa_weights[i] = (self.swa_weights[i] * 
                    (epoch - self.swa_epoch) + self.model.get_weights()[i])/((epoch - self.swa_epoch)  + 1)  
        else:
            pass
        
    def on_train_end(self, logs=None):
        self.model.set_weights(self.swa_weights)
        print('Final model parameters set to stochastic weight average.')
        self.model.save_weights(self.filepath)
        print('Final stochastic averaged weights saved to file.')

swa=SWA('/content/drive/My Drive/Vehicle Janata Hack/SWA_weights.h5',EPOCHS-3)
snapshot = SnapshotCallbackBuilder(nb_epochs=EPOCHS,nb_snapshots=1,init_lr=init_lr)


In [ ]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=1e-5)
chk=tensorflow.keras.callbacks.ModelCheckpoint(
                                                filepath='/content/drive/My Drive/Vehicle Janata Hack/chk_model.h5',
                                                monitor='val_loss',
                                               mode='min',
                                               save_best_only=True
                                               )
model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer=opt)
H=model.fit(data,
            train_labels,
            validation_data=(test_data,test_labels_1),
            epochs=EPOCHS,
            batch_size=32,
            callbacks=[chk]+snapshot.get_callbacks()
            )


In [ ]:
from tensorflow.keras.models import load_model
model.load_weights('/content/drive/My Drive/Vehicle Janata Hack/SWA_weights.h5')

In [ ]:
model=load_model('/content/drive/My Drive/Vehicle Janata Hack/chk_model.h5')

In [ ]:
test=pd.read_csv('/content/drive/My Drive/Vehicles Janata test.csv/test_vc2kHdQ.csv')

In [ ]:
img_names=test['image_names']

In [ ]:
lb.classes_

array(['no', 'yes'], dtype='<U3')

In [ ]:
results=[]
for i in img_names:
  imgpath=os.path.sep.join(['/content/train/images',i])
  img=cv2.imread(imgpath)
  img=preprocess_input(img)
  img=np.expand_dims(img,axis=0)
  out=model.predict(img)
  if(out[0][0]>out[0][1]):
    results.append(0)
  else:
    results.append(1)

In [ ]:
len(results)

706

In [ ]:
test_results=pd.Series(results,name='emergency_or_not')

In [ ]:
table=pd.concat([pd.Series(img_names,name='image_names'),test_results],axis=1)

In [ ]:
table.to_csv('/content/drive/My Drive/Vehicle Janata Hack/test_results.csv')